<a href="https://colab.research.google.com/github/Alice-zou/Natural-Disaster-Chatbot/blob/main/FineTune28_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the OpenAI library of version 0.28. Note, OpenAI lib has APIs changed version over versions.
!pip install openai==0.28

# Import the library of openai and others
import openai
import json
import time
import os
from google.colab import drive

# Mount Google Drive to the Colab environment
drive.mount('/content/drive')

# Define the path to the folder in Google Drive where the JSON file for dataset locates
drive_folder_path = '/content/drive/My Drive/Subsidence'

# Ensure the destination folder exists, if not create it
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
os.chdir(drive_folder_path)
!pwd

# Setup OpenAI API key so that it is able to invoke openai APIs
openai.api_key = 'Use ur Own Key'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/My Drive/Subsidence


In [ ]:
# Upload training data to OpenAI and check status. It may take a while depending on file size.
response = openai.File.create(
    file=open('/content/drive/My Drive/Subsidence/qa_output1.jsonl'),
    purpose='fine-tune'
)
training_file_id = response['id']
print(f"Training file ID: {training_file_id}")

status = response['status']

while status != 'processed':
        print(f"File status: {status}. Waiting for the file to be processed...")
        time.sleep(3)  # Wait for 10 seconds
        file_response = openai.File.retrieve(training_file_id)
        status = file_response['status']
        print(file_response)


Training file ID: file-BWGfVFvz2Lm0pW4RhUAcckP3


In [ ]:
# Create a job to fine-tune the model
fine_tuning_response = openai.FineTuningJob.create(training_file=training_file_id, model='gpt-3.5-turbo')
print(fine_tuning_response)
print("Starting fine tuning job with ID: " + fine_tuning_response['id'])

# View the fine-tune job status. It lists 10 jobs in the history. The first one is the current one. It may take a while to finish the job.
print(openai.FineTuningJob.list(limit=10))

{
  "object": "fine_tuning.job",
  "id": "ftjob-aK21qAzKB4xh26MwRfC0Jtkl",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1719209392,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-JHwka2okZO9VGaLoJJGb81ec",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-BWGfVFvz2Lm0pW4RhUAcckP3",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 1023672051,
  "estimated_finish": null,
  "integrations": []
}
Starting fine tuning job with ID: ftjob-aK21qAzKB4xh26MwRfC0Jtkl
{
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job",
      "id": "ftjob-aK21qAzKB4xh26MwRfC0Jtkl",
      "model": "gpt-3.5-turbo-0125",
      "created_at": 1719209392,
      "finished_at": null,
      "fine_tuned_model": null,
      "organization_id": "org-JHwka2okZO9VGaLoJJGb

In [ ]:

# Get the fine-tune job ID
fine_tune_id = fine_tuning_response['id']
print(f"Fine-tune job ID: {fine_tune_id}")

# Keep monitoring the fine-tuning job until it finishes.
while True:
    fine_tune_status = openai.FineTuningJob.retrieve(id=fine_tune_id)
    status = fine_tune_status['status']
    print(f"Fine-tune job status: {status}")
    if status == 'succeeded':
        fine_tuned_model = fine_tune_status['fine_tuned_model']
        print(f"Fine-tuning completed. Model: {fine_tuned_model}")
        break
    elif status == 'failed':
        print("Fine-tuning failed")
        break
    time.sleep(30)




Fine-tune job ID: ftjob-aK21qAzKB4xh26MwRfC0Jtkl
Fine-tune job status: validating_files
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: running
Fine-tune job status: succeeded
Fine-tuning completed. Model: ft:gpt-3.5-turbo-0125:personal::9dXIXTaF


In [ ]:
# Use the above fine-tuned model to chat
response = openai.ChatCompletion.create(
    model=fine_tuned_model,
    messages=[
       {"role": "system", "content": "You are a helpful chatbot that tries to provide accurate answers to questions regarding natual disasters."},
       {"role": "user", "content": "How to predict a subsidence disaster?"}
  ]
)
print(response['choices'][0].message)

{
  "role": "assistant",
  "content": " Subsidence disasters can be predicted through the monitoring and analysis of ground movement patterns using technology such as satellite radar."
}
